In [5]:
import pickle
import sympy
import pandas as pd
import sys
sys.path.append('../../')
import cartpole_error_monitoring

def load_from_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        F = data['F']
        lmd_time_series = data['lmd_time_series']
        state_time_series = data['state_time_series']
    return F, lmd_time_series, state_time_series

F, lmd_time_series, state_time_series = load_from_file('symbolic_F.pkl')


In [6]:
for i in range(F.shape[1]):
    variables = set()
    matrix = F[i,0]
    variables.update(matrix.free_symbols)
    # print(list(variables))

for i in range(state_time_series.shape[0]):
    for j in range(state_time_series.shape[1]):
        variables = set()
        matrix = state_time_series[i,j]
        variables.update(matrix.free_symbols)
        # print(list(variables))

for i in range(lmd_time_series.shape[0]):
    for j in range(lmd_time_series.shape[1]):
        variables = set()
        matrix = lmd_time_series[i,j]
        variables.update(matrix.free_symbols)
        # print(list(variables))


In [7]:
# 読み込みたいファイル名のリスト
filenames = ['generated_code_uopt.log', 'generated_code_x.log', 'generated_code_t.log']

# 各ファイルを読み込み、その結果をリストに保存します
df_list = [pd.read_csv(filename, delim_whitespace=True, header=None) for filename in filenames]

# 各データフレームをnumpyの行列に変換し、それをリストに保存します
u_matrix, x_matrix, t = [df.values for df in df_list]


In [8]:
from math import sqrt

from matplotlib import pyplot as plt


config = {
    "nx": 4,  # Number of states
    "nu": 1,  # Number of control inputs
    "ocp_name": "generated_code",  # Name of the optimal control problem
    "Tf": 2.0,  # Time horizon for the optimal control problem
    "alpha": 0.0,  # Soft horizon parameter
    "sampling_time": 0.001,  # Sampling time for the discrete-time optimal control problem
    "N": 5,  # Number of discretized intervals
    "finite_difference_epsilon": 1.0e-08,  # Epsilon for finite difference approximations
    "zeta": 1000,  # Penalty parameter
    "kmax": 5,  # Maximum number of iterations for the semi-smooth Newton method
    "initial_time": 0,  # Initial time for the simulation
    "initial_state": [0, 0, 0, 0],  # Initial state for the simulation
    "simulation_length": 10,  # Length of the simulation in seconds
    "solution_initial_guess": [
        0.01
    ],  # Initial guess for the optimal control problem solution
    "tolerance": 1.0e-06,  # Tolerance for the optimal control problem solution
    "max_iterations": 50,  # Maximum number of iterations for the optimal control problem solution
}

# Create and configure the AutoGenU object
auto_gen_u = cartpole_error_monitoring.create_autogenu_object(config)
cartpole_error_monitoring.set_parameters(auto_gen_u, config)


["m_c", 2], ["m_p", 0.2], ["l", 0.5], ["g", 9.80665]
x = sympy.Matrix(sympy.symbols(f'x0:{config["nx"]}'))  # Initial state as variables
u = sympy.Matrix([sympy.symbols(f'u{i}_{j}') for i in range(config["nu"]) for j in range(config["N"])]).reshape(config["nu"], config["N"])
r = sympy.symbols('r[0:%d]' %(config["nu"]))
q_terminal = sympy.symbols('q_terminal[0:%d]' %(config["nx"]))
q = sympy.symbols('q[0:%d]' %(config["nx"]))
x_ref = sympy.symbols('x_ref[0:%d]' %(config["nx"]))


sub_param = auto_gen_u.make_subs_dicts()

# Create a list to store the L2 norms and F values
l2_norms = []
F_param_substituted = []

simulation_steps = int(config["simulation_length"]/config["sampling_time"])
print("simulation step:", simulation_steps)

# for k in range(int(config["simulation_length"]/config["sampling_time"])):
for k in range(5):
    print(k)
    # Create the substitutions for the state and input at the current time step
    subs_x = [(x[l], x_matrix[k,l]) for l in range(config["nx"])]
    subs_u = [(u[l], u_matrix[k,l]) for l in range(config["N"])]

    # Create a list to store the F values for the current time step
    F_param_substituted_step = []

    # Loop over the control inputs
    norm = 0
    for i in range(config["N"]):
        for j in range(config["nu"]):
            # Calculate the value of F
            F_val = F[i,j].subs(sub_param).subs(subs_x).subs(subs_u)

            F_param = F[i,j].subs(sub_param)
            F_param_substituted_step.append(F_param)
            
            # Add to the squared L2 norm
            norm += F_val**2

    
    # if k%50 == 0:
    #     print(k)
    # Append the square root of the norm (i.e., the L2 norm) to the list
    l2_norms.append(sqrt(norm))

    # Append the F values for the current step to the list
    F_param_substituted.append(F_param_substituted_step)

# Plot the L2 norm of F
plt.plot(l2_norms,t[0,k])
plt.ylabel(r'$\Vert F \Vert_2$')

simulation step: 10000
0
0
1


In [ ]:
from sympy import Matrix
from sympy.parsing.sympy_parser import parse_expr

# F_matrix = Matrix(config["N"], config["nu"], F_param_substituted)
F = F_param_substituted[0][:]

# for i in range(config["N"]):
#     for j in range(config["nu"]):
#         variables = set()
#         matrix = F_param_substituted[i][j]
#         variables.update(matrix.free_symbols)
#         print(list(variables))

for i in range(1):
    F_matrix = Matrix(config["N"], config["nu"], F_param_substituted[i][:])
    print(F_matrix)

# for i in range(config["N"]):
#     for j in range(config["nu"]):
#         variables = set()
#         matrix = F_param_substituted[i][j]
#         variables.update(matrix.free_symbols)
#         print(list(variables))




Matrix([[u0_0 + (0.0177777777777778*x0 + 23.357037037037*x2 + 15.5634567901235*(u0_0 + 0.2*(0.5*x1**2 + 9.80665*cos(x1))*sin(x1))/(0.2*sin(x1)**2 + 2) + 11.1111111111111*(u0_1 + 0.2*(0.5*(x1 + 0.666666666666667*x3)**2 + 9.80665*cos(x1 + 0.666666666666667*x3))*sin(x1 + 0.666666666666667*x3))/(0.2*sin(x1 + 0.666666666666667*x3)**2 + 2) - 23.3333333333333)/(0.2*sin(x1)**2 + 2) - 2.0*(0.0177777777777778*x1 + 0.047037037037037*x3 - 0.222222222222222*(u0_2 + 0.2*(0.888888888888889*(0.75*x1 + x3 + 0.666666666666667*(-u0_0*cos(x1) - 0.1*x1**2*sin(x1)*cos(x1) - 21.57463*sin(x1))/(0.2*sin(x1)**2 + 2))**2 + 9.80665*cos(x1 + 1.33333333333333*x3 + 0.888888888888889*(-u0_0*cos(x1) - 0.1*x1**2*sin(x1)*cos(x1) - 21.57463*sin(x1))/(0.2*sin(x1)**2 + 2)))*sin(x1 + 1.33333333333333*x3 + 0.888888888888889*(-u0_0*cos(x1) - 0.1*x1**2*sin(x1)*cos(x1) - 21.57463*sin(x1))/(0.2*sin(x1)**2 + 2)))*(2*x2 + 1.33333333333333*(u0_0 + 0.2*(0.5*x1**2 + 9.80665*cos(x1))*sin(x1))/(0.2*sin(x1)**2 + 2) + 1.33333333333333*(u